In [90]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from scipy.optimize import minimize
from fredapi import Fred

In [91]:
tickers = ["SPY", "BND", "GLD", "QQQ", "VTI"]

In [92]:
end_date = datetime.today()
print(end_date)

2024-07-13 16:37:34.931373


In [93]:
start_date = end_date - pd.to_timedelta(5 * 365, 'D')
print(start_date)

2019-07-15 16:37:34.931373


In [94]:
adj_close_df = pd.DataFrame()

for ticker in tickers:
    data = yf.download(ticker, start = start_date, end = end_date)
    adj_close_df[ticker] = data["Adj Close"]

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

/Users/tina/anaconda3/envs/env_py_31011/lib/python3.10/site-packages/yfinance/utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/Users/tina/anaconda3/envs/env_py_31011/lib/python3.10/site-packages/yfinance/utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

/Users/tina/anaconda3/envs/env_py_31011/lib/python3.10/site-packages/yfinance/utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/Users/tina/anaconda3/envs/env_py_31011/lib/python3.10/site-packages/yfinance/utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')



[*********************100%%**********************]  1 of 1 completed


/Users/tina/anaconda3/envs/env_py_31011/lib/python3.10/site-packages/yfinance/utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [95]:
print(adj_close_df)

                   SPY        BND         GLD         QQQ         VTI
Date                                                                 
2019-07-15  277.972565  72.419449  133.529999  188.072922  141.750183
2019-07-16  277.076050  72.323082  132.399994  187.104233  141.390076
2019-07-17  275.190552  72.542122  134.580002  186.183960  140.457458
2019-07-18  276.198029  72.673531  136.479996  186.397079  140.974548
2019-07-19  274.663757  72.594650  134.470001  185.031189  140.171204
...                ...        ...         ...         ...         ...
2024-07-08  555.280029  72.400002  218.190002  497.339996  272.489990
2024-07-09  555.820007  72.370003  218.559998  497.769989  272.420013
2024-07-10  561.320007  72.430000  219.360001  502.959991  275.059998
2024-07-11  556.479980  72.790001  223.250000  491.929993  273.829987
2024-07-12  559.989990  72.980003  223.110001  494.820007  275.640015

[1258 rows x 5 columns]


In [98]:
log_returns = np.log(adj_close_df / adj_close_df.shift(1))

In [102]:
log_returns.dropna(inplace=True)
log_returns

,SPY,BND,GLD,QQQ,VTI
Date,,,,,
2019-07-16,-0.003230,-0.001332,-0.008499,-0.005164,-0.002544
2019-07-17,-0.006828,0.003024,0.016331,-0.004931,-0.006618
2019-07-18,0.003654,0.001810,0.014019,0.001144,0.003675
2019-07-19,-0.005570,-0.001086,-0.014837,-0.007355,-0.005715
2019-07-22,0.002453,0.001086,-0.000149,0.007927,0.002369
...,...,...,...,...,...
2024-07-08,0.001153,0.000000,-0.012480,0.002375,0.001359
2024-07-09,0.000972,-0.000414,0.001694,0.000864,-0.000257
2024-07-10,0.009847,0.000829,0.003654,0.010373,0.009644


In [106]:
# Covariance matrix
cov_matrix = log_returns.cov() * 252
print(cov_matrix)

          SPY       BND       GLD       QQQ       VTI
SPY  0.044019  0.002959  0.003927  0.049619  0.045064
BND  0.002959  0.004835  0.003906  0.003583  0.003155
GLD  0.003927  0.003906  0.023369  0.005238  0.004159
QQQ  0.049619  0.003583  0.005238  0.064476  0.050733
VTI  0.045064  0.003155  0.004159  0.050733  0.046511


In [107]:
def standard_deviation(weights, cov_matrix):
    variance = weights.T @ cov_matrix @ weights
    return np.sqrt(variance)

In [111]:
def expected_return(weights, log_returns):
    return np.sum(log_returns.mean() * weights) * 252

$$
SharpeRatio = {portfolioReturn - riskFreeRate \over portfolioStd}
$$

In [112]:
def sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate):
    return (expected_return(weights, log_returns) - risk_free_rate) / standard_deviation(weights, cov_matrix)

In [113]:
# Define the ticker symbol for the 10-year Treasury bond yield
ticker = "^TNX"

# Fetch the data
ten_year_treasury = yf.Ticker(ticker)

# Get the historical market data (we'll just fetch the latest data point)
#hist = ten_year_treasury.history(start=end_date, end=end_date)
hist = ten_year_treasury.history(period="300d")

# Extract the latest closing price (which represents the yield)
risk_free_rate = hist['Close'].iloc[-1] / 100
risk_free_rate

/Users/tina/anaconda3/envs/env_py_31011/lib/python3.10/site-packages/yfinance/utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


0.04189000129699707

In [120]:
def neg_sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate):
    return -sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate)

In [126]:
constraints = {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}
bounds = [(0, 0.4) for _ in range(len(tickers))]

In [127]:
initial_weights = np.repeat(1/len(tickers), len(tickers))
print(initial_weights)

[0.2 0.2 0.2 0.2 0.2]


In [128]:
optimize_results = minimize(neg_sharpe_ratio, initial_weights, args = (log_returns, cov_matrix, risk_free_rate), method = 'SLSQP',
                            constraints=constraints, bounds=bounds)

In [129]:
optimal_weights = optimize_results.x

In [130]:
print("Optimal weights:")
for ticker, weight in zip(tickers, optimal_weights):
    print(f"{ticker}: {weight:.4f}")

print()

optimal_portfolio_return = expected_return(optimal_weights, log_returns)
optimal_portfolio_volatility = standard_deviation(optimal_weights, cov_matrix)
optimal_sharpe_ratio = sharpe_ratio(optimal_weights, log_returns, cov_matrix, risk_free_rate)

print(f"Expected Annual Return: {optimal_portfolio_return:.4f}")
print(f"Expected Portfolio Volatility: {optimal_portfolio_volatility:.4f}")
print(f"Expected Sharpe ratio: {optimal_sharpe_ratio:.4f}")

Optimal weights:
SPY: 0.2000
BND: 0.0000
GLD: 0.4000
QQQ: 0.4000
VTI: 0.0000

Expected Annual Return: 0.1468
Expected Portfolio Volatility: 0.1614
Expected Sharpe ratio: 0.6500
